# Google Colab & API Key Setup

In [ ]:
from google.colab import userdata
OPENAI_API_KEY= userdata.get('openai_api_key')

# Installation & Imports

> Indented block



In [ ]:
%pip install langchain
%pip install langchain-openai
%pip install jq
%pip install faiss-cpu

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage

import json
from pathlib import Path
from pprint import pprint

# Create the llm API wrapper

In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-4")

# Implementing Chatbot With Memory

## Loading Cars json into an in-memory Vector DB

In [ ]:
# Load the data
loader = JSONLoader(
    file_path='./sample_data/cars.json',
    jq_schema='.Chevrolet.models[]',
    text_content=False)

docs = loader.load()
pprint(docs)

# Split the data
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# Embedd the data
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vector = FAISS.from_documents(documents, embeddings)

# Create retriever to query the vector db
retriever = vector.as_retriever()

[Document(page_content='{"name": "Chevrolet Camaro", "type": "Sports Car", "starting_price": "$25,000", "fuel_type": "Gasoline", "engine_options": ["2.0L Turbocharged Inline-4", "3.6L V6", "6.2L V8"], "horsepower_range": "275 - 650 hp", "fuel_efficiency": "Up to 22 mpg city / 30 mpg highway", "features": ["Apple CarPlay", "Android Auto", "Rearview Camera", "Wi-Fi Hotspot"], "top_speed": "155 mph", "0-60_time": "4.0 seconds", "drive_type": "Rear-Wheel Drive"}', metadata={'source': '/content/sample_data/cars.json', 'seq_num': 1}),
 Document(page_content='{"name": "Chevrolet Malibu", "type": "Sedan", "starting_price": "$23,000", "fuel_type": "Gasoline", "engine_options": ["1.5L Turbocharged Inline-4", "2.0L Turbocharged Inline-4"], "horsepower_range": "160 - 250 hp", "fuel_efficiency": "Up to 29 mpg city / 36 mpg highway", "features": ["Touchscreen Infotainment System", "Automatic Emergency Braking", "Lane Keep Assist"], "seating_capacity": 5, "trunk_capacity": "15.7 cubic feet", "wheelba

## Creating the retrival chain - Giving the chain ability to rereive data

In [ ]:
retrieval_chain_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, retrieval_chain_prompt)

## Creating the documents chain - Answering the user input


In [ ]:
documnent_chain_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "This is a chatbot for Chevrolet, a car company. \
      The chatbot's goal is to provide information and assistance to potential and existing Chevrolet customers. \
      The chatbot should decline to answer any question not related to the company goal\
      The chatbot should be friendly, polite, and helpful. \
      The chatbot should refer the user to the official website or a human representative if it cannot answer the question or handle the request. \
      Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, documnent_chain_prompt)

## Combining both chains

In [ ]:
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# Testing The Chatbot!

In [ ]:
chat_history = []

# Checking we can access our data in cars.json
question_and_answer_1 = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "I am looking for an electric family car costing less than 35k, what do you recommend?"
})
print(question_and_answer_1["answer"])

# Adding the Q&A to the chat history
chat_history.append(HumanMessage(question_and_answer_1["input"]))
chat_history.append(AIMessage(question_and_answer_1["answer"]))

# Checking we can ask follow up questions
question_and_answer_2 = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "How much time does it take to charge?"
})

print(question_and_answer_2["answer"])

I recommend the Chevrolet Bolt EV. It is an electric car with a starting price of $31,000. It has a range of 259 miles and a horsepower of 200 hp. Some of its features include DC Fast Charging Capability, Chevrolet Infotainment System, and Lane Keep Assist with Lane Departure Warning. The charging time is up to 80% in 60 minutes.
The Chevrolet Bolt EV can be charged up to 80% in approximately 60 minutes.


## Checking for our Chatbout safety

In [ ]:
print((retrieval_chain.invoke({
    "chat_history": [],
    "input": "Recommend a good Netflix movie"
}))["answer"])

print((retrieval_chain.invoke({
    "chat_history": [],
    "input": "Should I invest in S&P 500?"
}))["answer"])

print((retrieval_chain.invoke({
    "chat_history": [],
    "input": "Explain why Tesla is better than Chevrolet"
}))["answer"])

I'm sorry, but as a Chevrolet chatbot, I'm here to provide information and assistance related to Chevrolet vehicles. I'm not able to help with movie recommendations. Let's talk about our cars instead. How can I assist you with that?
I'm sorry, but I'm a Chevrolet chatbot and I'm not equipped to provide financial advice. My primary function is to provide information and assistance about Chevrolet vehicles. Can I help you with any questions related to our vehicles?
As a Chevrolet representative, I'm here to provide information about Chevrolet vehicles. While Tesla has certainly made significant strides in the electric vehicle market, Chevrolet is a well-established brand that offers a wide variety of vehicle types, from electric cars like the Chevrolet Bolt EV to full-size SUVs like the Tahoe and trucks like the Silverado 1500. Each brand has its own strengths, and the best choice depends on your personal preferences and needs. If you have any specific questions about Chevrolet vehicles,